In [3]:
%%capture
# install chromium, its driver, and selenium!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

In [4]:
from google.colab import drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/DataMan_Songs/Dataset/'

Mounted at /content/drive


In [5]:
import pandas as pd
import re
from collections import OrderedDict

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import requests
import time


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
df = pd.read_csv(dataset_path + '2_Dataset_SpotifyFeatures.csv')
print('Lenght Original dataset: ', len(df))

# DF = pd.read_csv(dataset_path + '1_Dataset_Song_Features_clean.csv')
# df_merge = DF.merge(df, on = ['Band', 'Song'], how='inner')
# print(len(df_merge))
# ## There are some songs with same name and different difficulty level
# df_merge[df_merge.duplicated() == True]
# df_merge.drop_duplicates(subset=['ID_TRACK'], inplace=True)
# print('Lenght dataset with songs file: ', len(df_merge))
# # df_merge.to_csv(path + 'Dataset/Dataset_with_fileaudioTrack.csv', index=False)

Lenght Original dataset:  1807


In [8]:
## https://www.famousbirthdays.com/bands/{Bandname}.html 
## div class='row' ---> per ogni elemento: salva il testo, apri il link, salva div class='person title' testo
##                                         (Name, Surname),            , (instrument played)

band_list = df['Band'].unique()

band_dict = {}
def band_info(band_name):

    print('-------------------------------------------------------')

    global band_dict
    band = {}

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # open it, go to a website, and get results
    wd = webdriver.Chrome(options=options)
    wd.get("https://www.famousbirthdays.com/")

    ## To fill: <input autocomplete="off" id="main-search" type="search" name="q" 
    ## placeholder=" Search " class="form-control search search-bar" aria-label="Enter Celebrity or Date" maxlength="80">

    ## Use search_box
    input  = wd.find_element('id', "main-search")
    input.send_keys("{} (band)".format(band_name))
    button = wd.find_element('xpath', '//*[@id="fb_search"]/button').click()

    main_site = wd.current_url
    print(main_site)

    ## Scrape page site
    page = requests.get(main_site)
    ## Avoid too many requests error
    if page.status_code == 429:
      time.sleep(int(page.headers["Retry-After"]))
    
    print(page)
    soup1 = BeautifulSoup(page.content, "html.parser")
    names= soup1.find_all('div', class_='name')
    for artist_name in names:  
        name = artist_name.text.split(', ')[0]
        name = name.replace('\n', '')
        try:
          name = name.replace(' ', '')
        except:
          continue
        
        band[name] = '' 
        # print(name)

    ## Add information about instruments played by each member of the band
    instruments = []
    soup = soup1.find_all('a', class_='face person-item clearfix')
    for artist in soup:
        artist_link = artist['href']
        print(artist_link)
        artist_page = requests.get(artist_link)
        artist_soup = BeautifulSoup(artist_page.content, "html.parser")
        play = artist_soup.find('div', class_='person-title')
        instruments.append(play.text.replace('\n ', '').replace('\n', ''))

    ## Add informations to the right dictionary and delete others useless keys
    n = 0
    for name in band.keys():
        n += 1
        try:
            band[name] = instruments[n-1]
        except:
            break

    band = {v: k for v, k in band.items() if k != ''}

    band_dict[band_name] = band
    time.sleep(1)

    return(print('{} info added'.format(band_name)))


In [ ]:
for el in band_list:
  try:  
    band_info(el)
  except:
    continue

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [2]:
band_dict1 = band_dict.copy()
band_dict1

NameError: ignored

In [ ]:
for key in band_dict.keys():
    # print(band_dict1[key])
    values = [[re.sub('\w+ ', '', x), y] for y, x in band_dict[key].items()]
    band_dict1[key] = {}
    for couple in values:
        band_dict1[key].setdefault(couple[0], []).append(couple[1])
    
    band_dict1[key] = OrderedDict(band_dict1[key])
    # print(band_dict1[key])
          
    # Take the first 5 artist......how to know how many drummers in a band?
    key_order = ['Singer', 'Guitarist', 'Bassist', 'Drummer', 'Pianist'] # the order you want
    for k in key_order: # a loop to force the order you want
        try:
            band_dict1[key].move_to_end(k) 
        except:
            continue
    
    band_dict1[key] = dict(band_dict1[key])
    # print(band_dict1[key])

df = pd.DataFrame.from_dict({(i): [band_dict1[i][j] for j in band_dict1[i].keys()] 
                           for i in band_dict1.keys() 
                           for j in band_dict1[i].keys()
                           },
                       orient='index',
)
df.head(3)

,0,1,2,3,4,5
AC/DC,"[BrianJohnson, BonScott(1946-1980)]","[AngusYoung, MalcolmYoung(1953-2017), MarkEvans]",[CliffWilliams],"[PhilRudd, ChrisSlade, SimonWright]",None,None
Led Zeppelin,[RobertPlant],[JimmyPage],[JohnPaulJones],[JohnBonham(1948-1980)],None,None
Kansas,[SteveWalsh],"[SteveMorse, KerryLivgren]","[BillyGreer, DaveHope]",[PhilEhart],None,None


In [ ]:
df = df.rename(columns={0: 'Singer', 
                   1: 'Guitarist', 
                   2: 'Bassist', 
                   3: 'Drummer', 
                   4: 'Pianist'})
df = df.iloc[: , :5]
df.to_csv(dataset_path + '3_Info_band.csv')